In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/develop/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os

/home/zulqarnain/.conda/envs/htfatorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/media/babak/74e696ad-c235-46fc-a6e7-1830e44c8a1b/zulqarnain/AffVids_mcwm/motion_corrected/'

task_log_csvs = 'editedlogfiles'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub%02d_run%02d_mc_MNI_masked.nii.gz'

AFFVIDS_FILE = 'data/affvids2018_mc_zrest.db'

def affvid_filename(subject, run):
    return affvids_dir + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round(t + TASK_ONSET_DELAY)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[9])
        end_time = float(cols[10])
        run = int(cols[1])
        fear_rating = float(cols[11])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
bad_runs[3] = [1, 2, 3]
bad_runs[7] = [3]
bad_runs[14] = [3]
bad_runs[22] = [1, 2, 3]
bad_runs[23] = [1]
bad_runs[24] = [1, 2, 3]
bad_runs[26] = [3]

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().split(' ')
        subject = int(header[4])
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(1, 4)}
        rest_ends_dict = {key: [] for key in range(1, 4)}
        for (i,rest) in enumerate(rest_lines):
            if rest.run in bad_runs[subject]:
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject]:
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True,zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(AFFVIDS_FILE) or OVERRIDE:
    affvids_db = niidb.FMriActivationsDb(AFFVIDS_FILE, mask='/media/babak/74e696ad-c235-46fc-a6e7-1830e44c8a1b/zulqarnain/AffVids_mcwm/wholebrain2.nii.gz',
                                         smooth=6)

    for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            affvids_db.upsert(block)

01/22/2020 11:05:47 Subject 10
01/22/2020 11:05:47 Block 0 rest of run 1 started at 0.000000, ended at 24.000000
01/22/2020 11:05:47 Block 1 rest of run 2 started at 0.000000, ended at 22.000000
01/22/2020 11:05:47 Block 2 rest of run 3 started at 0.000000, ended at 22.000000
01/22/2020 11:05:47 Block 3 spider_low_3.m4v of run 2 started at 23.000000, ended at 44.000000
01/22/2020 11:05:47 Block 4 heights_low_6.m4v of run 3 started at 23.000000, ended at 41.000000
01/22/2020 11:05:47 Block 5 spider_high_5.m4v of run 1 started at 25.000000, ended at 46.000000
01/22/2020 11:05:47 Block 6 rest of run 3 started at 42.000000, ended at 63.000000
01/22/2020 11:05:47 Block 7 rest of run 2 started at 45.000000, ended at 68.000000
01/22/2020 11:05:47 Block 8 rest of run 1 started at 47.000000, ended at 69.000000
01/22/2020 11:05:47 Block 9 heights_high_2.mov of run 3 started at 64.000000, ended at 85.000000
01/22/2020 11:05:47 Block 10 heights_high_1.m4v of run 2 started at 69.000000, ended at 89

01/22/2020 11:05:47 Block 15 social_high_3.mov of run 2 started at 110.000000, ended at 130.000000
01/22/2020 11:05:47 Block 16 spider_low_3.m4v of run 1 started at 112.000000, ended at 133.000000
01/22/2020 11:05:47 Block 17 spider_high_3.mov of run 3 started at 113.000000, ended at 133.000000
01/22/2020 11:05:47 Block 18 rest of run 2 started at 131.000000, ended at 152.000000
01/22/2020 11:05:47 Block 19 rest of run 1 started at 134.000000, ended at 157.000000
01/22/2020 11:05:47 Block 20 rest of run 3 started at 134.000000, ended at 154.000000
01/22/2020 11:05:47 Block 21 heights_high_4.m4v of run 2 started at 153.000000, ended at 173.000000
01/22/2020 11:05:47 Block 22 social_high_6.mov of run 3 started at 155.000000, ended at 176.000000
01/22/2020 11:05:47 Block 23 spider_high_1.mov of run 1 started at 158.000000, ended at 178.000000
01/22/2020 11:05:47 Block 24 rest of run 2 started at 174.000000, ended at 196.000000
01/22/2020 11:05:47 Block 25 rest of run 3 started at 177.0000

01/22/2020 11:05:48 Block 29 heights_high_4.m4v of run 2 started at 200.000000, ended at 220.000000
01/22/2020 11:05:48 Block 30 rest of run 3 started at 217.000000, ended at 240.000000
01/22/2020 11:05:48 Block 31 rest of run 1 started at 221.000000, ended at 243.000000
01/22/2020 11:05:48 Block 32 rest of run 2 started at 221.000000, ended at 242.000000
01/22/2020 11:05:48 Block 33 spider_high_2.m4v of run 3 started at 241.000000, ended at 261.000000
01/22/2020 11:05:48 Block 34 heights_high_6.mov of run 2 started at 243.000000, ended at 264.000000
01/22/2020 11:05:48 Block 35 spider_low_4.m4v of run 1 started at 244.000000, ended at 265.000000
01/22/2020 11:05:48 Block 36 rest of run 3 started at 262.000000, ended at 285.000000
01/22/2020 11:05:48 Block 37 rest of run 2 started at 265.000000, ended at 285.000000
01/22/2020 11:05:48 Block 38 rest of run 1 started at 266.000000, ended at 286.000000
01/22/2020 11:05:48 Block 39 social_low_5.m4v of run 2 started at 286.000000, ended at 

01/22/2020 11:05:48 Block 43 rest of run 1 started at 307.000000, ended at 330.000000
01/22/2020 11:05:48 Block 44 rest of run 2 started at 311.000000, ended at 331.000000
01/22/2020 11:05:48 Block 45 spider_low_4.m4v of run 3 started at 327.000000, ended at 348.000000
01/22/2020 11:05:48 Block 46 social_high_1.mov of run 1 started at 331.000000, ended at 351.000000
01/22/2020 11:05:48 Block 47 spider_high_5.m4v of run 2 started at 332.000000, ended at 353.000000
01/22/2020 11:05:48 Block 48 rest of run 3 started at 349.000000, ended at 372.000000
01/22/2020 11:05:48 Block 49 rest of run 1 started at 352.000000, ended at 373.000000
01/22/2020 11:05:48 Block 50 rest of run 2 started at 354.000000, ended at 374.000000
01/22/2020 11:05:48 Block 51 spider_high_6.mov of run 3 started at 373.000000, ended at 392.000000
01/22/2020 11:05:48 Block 52 heights_low_6.m4v of run 1 started at 374.000000, ended at 393.000000
01/22/2020 11:05:48 Block 53 spider_low_5.mov of run 2 started at 375.000000

01/22/2020 11:05:49 Excluding block 62 rest of run 3 for motion
01/22/2020 11:05:49 Block 63 spider_high_5.m4v of run 1 started at 460.000000, ended at 481.000000
01/22/2020 11:05:49 Block 64 social_high_5.m4v of run 2 started at 461.000000, ended at 481.000000
01/22/2020 11:05:49 Excluding block 65 heights_high_6.mov of run 3 for motion
01/22/2020 11:05:49 Block 66 rest of run 1 started at 482.000000, ended at 502.000000
01/22/2020 11:05:49 Block 67 rest of run 2 started at 482.000000, ended at 505.000000
01/22/2020 11:05:49 Excluding block 68 rest of run 3 for motion
01/22/2020 11:05:49 Block 69 spider_high_6.mov of run 1 started at 503.000000, ended at 523.000000
01/22/2020 11:05:49 Block 70 social_low_6.m4v of run 2 started at 506.000000, ended at 525.000000
01/22/2020 11:05:49 Excluding block 71 heights_low_2.m4v of run 3 for motion
01/22/2020 11:05:49 Block 72 rest of run 1 started at 524.000000, ended at 0.000000
01/22/2020 11:05:49 Block 73 rest of run 2 started at 526.000000, 

...

01/22/2020 11:05:54 Block 27 social_high_6.mov of run 3 started at 198.000000, ended at 218.000000
01/22/2020 11:05:54 Block 28 heights_low_5.m4v of run 2 started at 199.000000, ended at 219.000000
01/22/2020 11:05:54 Block 29 spider_low_1.m4v of run 1 started at 200.000000, ended at 220.000000
01/22/2020 11:05:54 Block 30 rest of run 3 started at 219.000000, ended at 240.000000
01/22/2020 11:05:54 Block 31 rest of run 2 started at 220.000000, ended at 241.000000
01/22/2020 11:05:54 Block 32 rest of run 1 started at 221.000000, ended at 244.000000
01/22/2020 11:05:54 Block 33 social_low_2.mov of run 3 started at 241.000000, ended at 262.000000
01/22/2020 11:05:54 Block 34 spider_high_3.mov of run 2 started at 242.000000, ended at 262.000000
01/22/2020 11:05:54 Block 35 spider_high_6.mov of run 1 started at 245.000000, ended at 264.000000
01/22/2020 11:05:54 Block 36 rest of run 2 started at 263.000000, ended at 284.000000
01/22/2020 11:05:54 Block 37 rest of run 3 started at 263.000000

01/22/2020 11:05:55 Block 41 heights_high_2.mov of run 2 started at 288.000000, ended at 309.000000
01/22/2020 11:05:55 Block 42 rest of run 3 started at 306.000000, ended at 327.000000
01/22/2020 11:05:55 Block 43 rest of run 1 started at 307.000000, ended at 328.000000
01/22/2020 11:05:55 Block 44 rest of run 2 started at 310.000000, ended at 330.000000
01/22/2020 11:05:55 Block 45 spider_low_3.m4v of run 3 started at 328.000000, ended at 348.000000
01/22/2020 11:05:55 Block 46 spider_low_1.m4v of run 1 started at 329.000000, ended at 349.000000
01/22/2020 11:05:55 Block 47 heights_low_2.m4v of run 2 started at 331.000000, ended at 351.000000
01/22/2020 11:05:55 Block 48 rest of run 3 started at 349.000000, ended at 370.000000
01/22/2020 11:05:55 Block 49 rest of run 1 started at 350.000000, ended at 372.000000
01/22/2020 11:05:55 Block 50 rest of run 2 started at 352.000000, ended at 374.000000
01/22/2020 11:05:55 Block 51 heights_high_3.m4v of run 3 started at 371.000000, ended at 

01/22/2020 11:05:55 Block 55 rest of run 2 started at 395.000000, ended at 415.000000
01/22/2020 11:05:55 Block 56 rest of run 1 started at 396.000000, ended at 417.000000
01/22/2020 11:05:55 Block 57 social_low_3.m4v of run 2 started at 416.000000, ended at 437.000000
01/22/2020 11:05:55 Block 58 social_low_6.m4v of run 3 started at 417.000000, ended at 436.000000
01/22/2020 11:05:55 Block 59 heights_low_6.m4v of run 1 started at 418.000000, ended at 436.000000
01/22/2020 11:05:55 Block 60 rest of run 1 started at 437.000000, ended at 458.000000
01/22/2020 11:05:55 Block 61 rest of run 3 started at 437.000000, ended at 458.000000
01/22/2020 11:05:55 Block 62 rest of run 2 started at 438.000000, ended at 458.000000
01/22/2020 11:05:55 Block 63 heights_high_4.m4v of run 1 started at 459.000000, ended at 479.000000
01/22/2020 11:05:55 Block 64 social_low_1.m4v of run 2 started at 459.000000, ended at 480.000000
01/22/2020 11:05:55 Block 65 spider_high_4.m4v of run 3 started at 459.000000

01/22/2020 11:05:56 Subject 8
01/22/2020 11:05:56 Block 0 rest of run 1 started at 0.000000, ended at 22.000000
01/22/2020 11:05:56 Block 1 rest of run 2 started at 0.000000, ended at 22.000000
01/22/2020 11:05:56 Block 2 rest of run 3 started at 0.000000, ended at 22.000000
01/22/2020 11:05:56 Block 3 heights_low_6.m4v of run 1 started at 23.000000, ended at 41.000000
01/22/2020 11:05:56 Block 4 spider_high_5.m4v of run 2 started at 23.000000, ended at 44.000000
01/22/2020 11:05:56 Block 5 heights_low_1.m4v of run 3 started at 23.000000, ended at 44.000000
01/22/2020 11:05:56 Block 6 rest of run 1 started at 42.000000, ended at 64.000000
01/22/2020 11:05:56 Block 7 rest of run 2 started at 45.000000, ended at 67.000000
01/22/2020 11:05:56 Block 8 rest of run 3 started at 45.000000, ended at 68.000000
01/22/2020 11:05:56 Block 9 heights_low_5.m4v of run 1 started at 65.000000, ended at 85.000000
01/22/2020 11:05:56 Block 10 spider_low_6.m4v of run 2 started at 68.000000, ended at 89.00

01/22/2020 11:05:56 Block 15 heights_low_2.m4v of run 2 started at 110.000000, ended at 131.000000
01/22/2020 11:05:56 Block 16 spider_high_4.m4v of run 1 started at 112.000000, ended at 132.000000
01/22/2020 11:05:56 Block 17 social_high_3.mov of run 3 started at 113.000000, ended at 133.000000
01/22/2020 11:05:56 Block 18 rest of run 2 started at 132.000000, ended at 152.000000
01/22/2020 11:05:56 Block 19 rest of run 1 started at 133.000000, ended at 156.000000
01/22/2020 11:05:56 Block 20 rest of run 3 started at 134.000000, ended at 155.000000
01/22/2020 11:05:56 Block 21 heights_high_6.mov of run 2 started at 153.000000, ended at 174.000000
01/22/2020 11:05:56 Block 22 spider_high_3.mov of run 3 started at 156.000000, ended at 176.000000
01/22/2020 11:05:56 Block 23 spider_low_5.mov of run 1 started at 157.000000, ended at 177.000000
01/22/2020 11:05:56 Block 24 rest of run 2 started at 175.000000, ended at 195.000000
01/22/2020 11:05:57 Block 25 rest of run 3 started at 177.0000

In [6]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', affvids_dir)

01/22/2020 11:05:57 Finished building NiiDb out of AffVids dataset in /media/babak/74e696ad-c235-46fc-a6e7-1830e44c8a1b/zulqarnain/AffVids_mcwm/motion_corrected/
